In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
import math
from typing import Callable, Tuple, List

# Langrange interpolation

In [2]:
def lagrange_polynomial(x_values:np.ndarray, y_values:np.ndarray) -> sp.Expr:
    n = len(x_values)
    x = sp.symbols('x')
    polynomial = 0
    for i in range(n):
        L = 1
        for j in range(n):
            if i != j:
                L *= (x - x_values[j])/(x_values[i] - x_values[j])
        polynomial += y_values[i] * L
    return polynomial

## Tests for Lagrange interpolation method

In [3]:
x = np.array([-1, 0, 2], np.float64)
y = np.array([4, 1, -1], np.float64)

polynomial = lagrange_polynomial(x, y)
polynomial.simplify()

0.666666666666667*x**2 - 2.33333333333333*x + 1.0

# Newton interpolation

In [4]:
def divided_difference(x_values:np.ndarray, y_values:np.ndarray) -> sp.Expr:
    n = len(x_values)
    if n == 1:
        return y_values[0]
    return (divided_difference(x_values[1:], y_values[1:]) - divided_difference(x_values[:-1], y_values[:-1]))/(x_values[-1] - x_values[0])

def newton_interpolation(x_values:np.ndarray, y_values:np.ndarray) -> sp.Expr:
    n = len(x_values)
    x = sp.symbols('x')
    polynomial = y_values[0]
    for i in range(1, n):
        L = 1
        for j in range(i):
            L *= (x - x_values[j])
        polynomial += L * divided_difference(x_values[:i+1], y_values[:i+1])
    return polynomial


# Tests for Newton interpolation

In [5]:
x = np.array([-1, 0, 2], np.float64)
y = np.array([4, 1, -1], np.float64)

polynomial = newton_interpolation(x, y)
polynomial.simplify()

0.666666666666667*x**2 - 2.33333333333333*x + 1.0

# Splines

In [6]:
class Spline:
    def __init__(self):
        self.curves = []
        
    def add_curve(self, curve : Callable[[float], float], x_lim_inf : float, x_lim_sup : float):
        left = 0
        right = len(self.curves) - 1
        while left <= right:
            mid = left + (right - left) // 2
            if self.curves[mid]['x_lim_inf'] == x_lim_inf:
                raise ValueError('A curve with the same x_lim_inf already exists')
            elif self.curves[mid]['x_lim_inf'] < x_lim_inf:
                left = mid + 1
            else:
                right = mid - 1

        self.curves.insert(left, {'curve': curve, 'x_lim_inf': x_lim_inf, 'x_lim_sup': x_lim_sup})

    def evaluate(self, x : float) -> float:
        # Find the curve that contains x by doing a binary search
        # if no such curve exists it will raise an error

        left = 0
        right = len(self.curves) - 1

        while left <= right:
            mid = left + (right - left) // 2
            if self.curves[mid]['x_lim_inf'] <= x <= self.curves[mid]['x_lim_sup']:
                # return self.curves[mid]['curve'].subs('x', x)
                return self.curves[mid]['curve'](x)
            elif self.curves[mid]['x_lim_inf'] < x:
                left = mid + 1
            else:
                right = mid - 1

        raise ValueError('No curve found for x')

In [7]:
x_values = np.array([1, 2, 5, 7], np.float64)
y_values = np.array([1, 2, 3, 25], np.float64)

n = len(x_values)

spline = Spline()

for i in range(n - 1):
    x0, x1 = x_values[i], x_values[i + 1]
    y0, y1 = y_values[i], y_values[i + 1]
    curve = lambda x, x0=x0, x1=x1, y0=y0, y1=y1: y0 + (y1 - y0)/(x1 - x0) * (x - x0)
    spline.add_curve(curve, x0, x1)

spline.evaluate(7)

25.0

In [26]:
def cubic_natural_curve(x_values:np.ndarray, y_values:np.ndarray) -> List[Callable[[float], float]]:
    n = len(x_values)
    h = x_values[1] - x_values[0]
    a = y_values[0]
    b = (y_values[1] - y_values[0])/h
    c = 0
    d = 0
    curves = []

    # First curve is a special case, it's a linear curve
    curve = lambda x, a=a, b=b, x0=x_values[0]: a + b * (x - x0)
    curves.append(curve)

    for i in range(1, n - 1):
        h = x_values[i] - x_values[i - 1]
        a = y_values[i - 1]
        b = (y_values[i] - y_values[i - 1])/h
        c = (y_values[i - 1] - 2 * y_values[i] + y_values[i + 1])/h**2
        d = (y_values[i] - y_values[i - 1])/(h**3)
        curve = lambda x, a=a, b=b, c=c, d=d, x0=x_values[i]: a + b * (x - x0) + c * (x - x0)**2 + d * (x - x0)**3
        curves.append(curve)
    return curves

In [28]:
x_values = np.array([3, 4.5, 7, 9], np.float64)
y_values = np.array([2.5, 1.0, 2.5, 0.5], np.float64)

curves = cubic_natural_curve(x_values, y_values)

spline = Spline()

for i, curve in enumerate(curves):
    spline.add_curve(curve, x_values[i], x_values[i + 1])

spline.evaluate(4.5)

KeyboardInterrupt: 

# Function adjustment

In [10]:
def linear_regression(x_values:np.ndarray, y_values:np.ndarray) -> Callable[[float], float]:
    x_mean = np.mean(x_values)
    y_mean = np.mean(y_values)
    x_diff = x_values - x_mean
    y_diff = y_values - y_mean
    m = np.sum(x_diff * y_diff)/np.sum(x_diff**2)
    c = y_mean - m * x_mean
    return lambda x, m=m, c=c: m * x + c

In [11]:
def exponential_regression(x_values:np.ndarray, y_values:np.ndarray, 
                          x_fun: Callable[[float], float] = lambda x: x,
                          y_fun: Callable[[float], float] = lambda y: y,
                          a_fun: Callable[[float], float] = lambda a: a, 
                          b_fun: Callable[[float], float] = lambda b: b) -> Callable[[float], float]:
    x_values = x_fun(x_values)
    y_values = y_fun(y_values)

    x_sum = np.sum(x_values)
    y_sum = np.sum(y_values)
    x_times_y_sum = np.sum(x_values * y_values)
    x_squared_sum = np.sum(x_values**2)

    a = (len(x_values) * x_times_y_sum - x_sum * y_sum)/(len(x_values) * x_squared_sum - x_sum**2)
    b = (y_sum - a * x_sum)/len(x_values)

    return (a_fun(a), b_fun(b))

In [12]:
x_values = np.array([1, 2, 3, 4], np.float64)
y_values = np.array([3, 5, 6, 8], np.float64)

identity_function = lambda x: x
a, b = exponential_regression(x_values, y_values, identity_function, np.log, identity_function, np.exp)

# y = b * exp(a * x)

y = lambda x, a=a, b=b: b * np.exp(a * x)

y(2.6)

5.344462061821694